# Plan

1. Research earthquakes, research common earthquake classification methods
2. Get oriented with data, see if we can find any patterns
3. Decide on ML or non-ML approach

## ML approach plan options:

CNN (convolutional neural network)
- feed the model all the different times in each dataset, labeled as "arrival" or "not arrival"
- use the relationship between the velocities of different time steps to form a model classifying "arrival" or not.

In [101]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from obspy import read
import math

In [3]:
dir_name = 'data/lunar/training'
catalog_name = 'apollo12_catalog_GradeA_final.csv'

In [5]:
# get catalog dataframe
catalog_df = pd.read_csv(os.path.join(dir_name + '/catalogs/', catalog_name))
catalog_df = catalog_df.drop(catalog_df.index[20]).reset_index(drop=True)
# get list of files
arrival_time_catalog = catalog_df['time_rel(sec)']
filename_catalog = catalog_df['filename']
# make_plot(filename_catalog, arrival_time_catalog)

In [4]:
# make the plot for a csv file
def make_plot(namelist, arrival_time):
  fig, ax = plt.subplots(5, 1, figsize=(10, 20))
  # go through the catalog
  for i in range(len(namelist.head())):
    # read the csv and put it into a dataframe
    df = pd.read_csv(os.path.join(dir_name + '/data/S12_GradeA', namelist[i] + '.csv'))
    # put both the times and data columns into respective lists
    times = np.array(df['time_rel(sec)'].tolist())
    data = np.array(df['velocity(m/s)'].tolist())
    # make a plot of time vs velocity
    ax[i].plot(times, data)
    ax[i].set_xlabel('Time (s)')
    ax[i].set_ylabel('Velocity (m/s)')
    # add a red line for the correct arrival time
    ax[i].axvline(x = arrival_time[i], color='red',label='Rel. Arrival')
    arrival_line = ax[i].axvline(x=arrival_time[i], c='red', label='Abs. Arrival')
    ax[i].legend(handles=[arrival_line])

In [ ]:
# make_plot(filename_catalog[2])

# Machine Learning Section - LR


pairs to be fixed:
7,8
21,22
35,36
47,48
66,67

# New Version

Version 3

In [103]:
# split a csv into windows
def makeWindows(df, arrival_times, b_bound = 4, f_bound = 8, jump = 50):
  arrival_indices = []
  for arrival_time in arrival_times:
    time_list = df['time_rel(sec)'].tolist()
    for i in range(len(time_list)):
      if math.isclose(time_list[i], arrival_time, abs_tol=0.1):
        arrival_indices += [i]
        break
  velocities = df['velocity(m/s)'].tolist()

  result = []
  for offset in range(0, jump):
    f_pointer = (b_bound + f_bound) * jump + offset
    first_arr = np.array(velocities[0: f_pointer:jump])
    #print(first_arr)
    first_arr = np.array([np.float64(abs(i)) for i in first_arr])
    result = result + [[first_arr, False]]
    f_pointer += jump


    while f_pointer < len(velocities):
      target = f_pointer - (f_bound * jump)
      classification = False
      if target in arrival_indices:
        classification = True
      # print(np.float64(velocities[f_pointer]))
      # print([np.array(result[-1][0][1:]) + np.float64(velocities[f_pointer])])
      result.append([np.concatenate((np.array(result[-1][0][1:]), np.float64(abs(velocities[f_pointer]))), axis=None), classification])
      f_pointer += jump

  return result

In [104]:
# create windows in a big dataframe for any number of csvs
def make_many_windows(count = catalog_df.shape[0], jumps = 50):
  windows_df = pd.DataFrame(columns=['window', 'classification'])
  # keep track of our duplicate csvs (multiple arrival times)
  duplicates = {7:8, 21:22, 35:36, 47:48, 66:67}
  # go through the amount of csv files we want to look at
  for i in range(count):
    if i in duplicates.values():
      continue

    arrival_times = [catalog_df.loc[i, 'time_rel(sec)']]
    if i in duplicates.keys():
      arrival_times.append(catalog_df.loc[duplicates[i], 'time_rel(sec)'])

    filename = catalog_df.loc[i, 'filename']
    csv_df = pd.read_csv(os.path.join(dir_name + '/data/S12_GradeA/', filename + '.csv'))
    new_windows = pd.DataFrame(makeWindows(csv_df, arrival_times, jump = jumps), columns=['window', 'classification'])
    windows_df = pd.concat([windows_df, new_windows])
  return windows_df



In [105]:
# cProfile.run("make_one_window()")
windows_df = make_many_windows(20, jumps=200)

In [106]:
temp_df = windows_df.query('classification == True')
print(temp_df.head(10))
print(temp_df.iloc[0,0])

                                                   window classification
392895  [2.615472186269719e-11, 1.0525445909367719e-10...           True
199928  [1.2546847794692858e-10, 3.3448070314031916e-1...           True
449875  [2.0969773143235136e-10, 2.6026914867671456e-1...           True
42908   [1.2199051835026662e-10, 3.4291220158383776e-1...           True
78684   [1.1956302898365138e-10, 1.0253514637434532e-1...           True
429779  [1.6301366254506448e-10, 9.883457199931022e-11...           True
563855  [2.1210412791239447e-10, 1.9693108929735575e-1...           True
100365  [4.076576634813371e-10, 2.595375292988274e-10,...           True
115406  [3.31970875557756e-10, 1.7570268736368482e-10,...           True
22332   [5.041614617857014e-10, 7.146081895328625e-11,...           True
[2.61547219e-11 1.05254459e-10 2.52200479e-10 1.18134078e-10
 6.72298991e-10 3.10715850e-11 1.40079832e-10 9.93506220e-10
 2.01385078e-09 2.61848696e-11 2.36421898e-09 9.48563512e-11
 6.13876848e-1

In [107]:
windows_df

,window,classification
0,"[6.153278962788711e-14, 2.85616945537678e-15, ...",False
1,"[2.85616945537678e-15, 1.730540858201767e-14, ...",False
2,"[1.730540858201767e-14, 3.3823113618422784e-13...",False
3,"[3.3823113618422784e-13, 2.319956473454342e-13...",False
4,"[2.319956473454342e-13, 3.441583532076547e-12,...",False
...,...,...
570022,"[7.572116065506919e-11, 1.945692252264205e-11,...",False
570023,"[1.945692252264205e-11, 9.572665287095862e-12,...",False
570024,"[9.572665287095862e-12, 5.697413589820812e-12,...",False
570025,"[5.697413589820812e-12, 2.8324028871074872e-11...",False


# Model Training

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [108]:
df = windows_df
df

,window,classification
0,"[6.153278962788711e-14, 2.85616945537678e-15, ...",False
1,"[2.85616945537678e-15, 1.730540858201767e-14, ...",False
2,"[1.730540858201767e-14, 3.3823113618422784e-13...",False
3,"[3.3823113618422784e-13, 2.319956473454342e-13...",False
4,"[2.319956473454342e-13, 3.441583532076547e-12,...",False
...,...,...
570022,"[7.572116065506919e-11, 1.945692252264205e-11,...",False
570023,"[1.945692252264205e-11, 9.572665287095862e-12,...",False
570024,"[9.572665287095862e-12, 5.697413589820812e-12,...",False
570025,"[5.697413589820812e-12, 2.8324028871074872e-11...",False


In [109]:
df = windows_df
df_zeros = df[df['classification'] == False]
df_zeros = df_zeros.sample(frac=0.0002, random_state=42)
df_ones = df[df['classification'] == True]
df_pruned = pd.concat([df_zeros, df_ones])
df_pruned
df = df_pruned
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.shape)
print(df)


(2186, 2)
                                                 window classification
0     [1.7768079150057295e-10, 4.6157111272474764e-1...          False
1     [2.231688784578763e-10, 1.3161729278831426e-10...          False
2     [8.658529926336031e-11, 1.0262497900244396e-10...          False
3     [9.648182630120286e-11, 1.6244328563800403e-10...          False
4     [2.1978817209464934e-10, 1.9663644773023595e-1...          False
...                                                 ...            ...
2181  [6.456444776974853e-10, 5.831839164890788e-10,...          False
2182  [2.2693860056226438e-10, 3.0142349712112284e-1...          False
2183  [1.4979905773453998e-10, 1.0119205295288019e-1...          False
2184  [4.994913032797356e-09, 1.140079810542627e-09,...          False
2185  [1.8831929285477692e-10, 3.1132164614629754e-1...          False

[2186 rows x 2 columns]


In [110]:
# Clean data for model training
X_clean = pd.DataFrame(df['window'].tolist())
X_clean = X_clean.dropna(axis = 1, how = 'any')
y = df['classification'].astype(int)
X_clean

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.776808e-10,4.615711e-10,2.752155e-12,1.006013e-10,5.560358e-11,1.473633e-10,2.227921e-11,2.006091e-10,2.397536e-11,9.178344e-11,2.367150e-10,1.047664e-10
1,2.231689e-10,1.316173e-10,3.161875e-10,6.489149e-11,2.440468e-10,1.819746e-10,1.763363e-10,6.598531e-10,1.090675e-11,1.407380e-10,3.676019e-11,5.152654e-10
2,8.658530e-11,1.026250e-10,6.141028e-11,8.671985e-11,1.637710e-10,2.923113e-10,4.898582e-11,4.435983e-10,3.388429e-10,2.914754e-11,3.354819e-10,3.861093e-11
3,9.648183e-11,1.624433e-10,2.872933e-11,1.967340e-10,1.931233e-10,2.648497e-10,1.947000e-10,2.611762e-10,9.707584e-11,2.277244e-10,5.590474e-10,8.711333e-12
4,2.197882e-10,1.966364e-10,1.433237e-10,3.245206e-10,4.306331e-11,4.352492e-10,2.874396e-11,3.472316e-10,1.760436e-10,6.188894e-11,7.807283e-11,3.330569e-10
...,...,...,...,...,...,...,...,...,...,...,...,...
2181,6.456445e-10,5.831839e-10,4.540541e-10,5.764002e-11,2.391920e-10,6.592754e-10,7.577688e-10,1.078497e-09,3.016015e-10,1.027445e-12,3.816238e-10,7.259305e-10
2182,2.269386e-10,3.014235e-11,1.420293e-10,1.184829e-10,1.041413e-10,3.119288e-10,1.086237e-10,3.495907e-11,1.099868e-10,5.647581e-10,1.874773e-10,9.964380e-11
2183,1.497991e-10,1.011921e-10,1.022454e-10,2.339914e-10,1.764276e-10,1.811897e-10,4.821468e-10,3.037316e-11,1.968186e-10,4.199692e-10,1.199541e-09,1.514998e-09
2184,4.994913e-09,1.140080e-09,4.058278e-10,3.150956e-09,3.700066e-09,5.243700e-10,2.882965e-10,1.571421e-09,1.503334e-09,2.883847e-09,2.155465e-09,9.148663e-10


In [111]:
class_weights = {0: 1, 1: 540000}
model = LogisticRegression(max_iter=100, class_weight='balanced')
model.fit(X_clean, y)

LogisticRegression(class_weight='balanced')

# Test Data Processing

### Using actual test data

In [112]:
test_dir_name = 'data/lunar/test/S12_GradeB/'

### Using portion of training data

In [113]:
# make same type of dataframe for testing data
def make_test_windows(start = 30, count = catalog_df.shape[0], jumps = 50):
  test_windows_df = pd.DataFrame(columns=['window', 'classification'])

  duplicates = {7:8, 21:22, 35:36, 47:48, 66:67}
  for i in range(start, count):
    if i in duplicates.values():
      continue

    arrival_times = [catalog_df.loc[i, 'time_rel(sec)']]
    if i in duplicates.keys():
      arrival_times.append(catalog_df.loc[duplicates[i], 'time_rel(sec)'])

    filename = catalog_df.loc[i, 'filename']
    csv_df = pd.read_csv(os.path.join(dir_name + '/data/S12_GradeA/', filename + '.csv'))
    new_windows = pd.DataFrame(makeWindows(csv_df, arrival_times, jump = jumps), columns=['window', 'classification'])
    test_windows_df = pd.concat([test_windows_df, new_windows])
  return test_windows_df

In [114]:
test_df = make_test_windows(start=30, count=catalog_df.shape[0], jumps=200)

In [115]:
test_df

,window,classification
0,"[1.406432122600214e-15, 1.8401397302403882e-17...",False
1,"[1.8401397302403882e-17, 1.5994157455475232e-1...",False
2,"[1.5994157455475232e-14, 8.791586538396387e-13...",False
3,"[8.791586538396387e-13, 3.356882446658291e-13,...",False
4,"[3.356882446658291e-13, 9.112072540232316e-13,...",False
...,...,...
570002,"[1.5372765341245761e-12, 4.313554191251328e-14...",False
570003,"[4.313554191251328e-14, 1.417290681164798e-13,...",False
570004,"[1.417290681164798e-13, 1.3318165700826157e-12...",False
570005,"[1.3318165700826157e-12, 5.069384195598557e-15...",False


### Prepare it for the model

In [113]:
import gc
gc.collect()

6707

In [116]:
X_test_clean = pd.DataFrame(test_df['window'].tolist())
X_test_clean = X_test_clean.dropna(axis = 1, how = 'any')
y_test = test_df['classification'].astype(int)

Run the model on some data

In [117]:
# Evaluation metrics
predictions = model.predict(X_test_clean)
print(predictions)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

confusion = confusion_matrix(y_test, predictions)
report = classification_report(y_test, predictions, zero_division=0)

print(confusion)
print(report)

[0 0 0 ... 0 0 0]
0.9999981020735227
[[23710041        0]
 [      45        0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00  23710041
           1       0.00      0.00      0.00        45

    accuracy                           1.00  23710086
   macro avg       0.50      0.50      0.50  23710086
weighted avg       1.00      1.00      1.00  23710086



In [118]:
# Examine how the model trains given a certain weight for class balancing
def test_weight(weight):
  model = LogisticRegression(max_iter=1000, solver='liblinear', class_weight = {0: 1, 1: weight})
  model.fit(X_clean, y)
  predictions = model.predict(X_test_clean)
  accuracy = accuracy_score(y_test, predictions)
  return accuracy

In [122]:
# Fine tuning 
def search_weights():
    l = 1000
    r = 5402340
    while True:
      mid = (r-l) // 2 + l
      print(mid)
      accuracy = test_weight(mid)
      print(accuracy)
      if accuracy >= 0.9999983091356555:
          l = mid
      elif accuracy <= 1.690864344489938e-06:
          r = mid
      else:
        print(accuracy)
        return mid

In [ ]:
search_weights()